<a href="https://colab.research.google.com/github/0715Swathi/MuleSOft_query/blob/main/Program3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import random
import csv

def g_0(n):
  return ("?",)*n
def s_0(n):
  return ("0",)*n

In [5]:
def more_general(h1,h2):
  more_general_parts=[]
  for x,y in zip(h1,h2):
    mg = x =="?" or (x!='0' and (x==y or y=='0'))
    more_general_parts.append(mg)

  return all(more_general_parts)

l1 = [1,2,3]
l2 = [3,4,5]
list(zip(l1,l2))

[(1, 3), (2, 4), (3, 5)]

In [6]:
def fulfills(examples,hypothesis):
  return more_general(hypothesis,examples)

In [7]:
def min_generalization(h,x):
  h_new = list(h)
  for i in range(len(h)):
    if not fulfills(x[i:i+1],h[i:i+1]):
      h_new[i] = '?' if h[i]!='0' else x[i]

  return [tuple(h_new)]

min_generalization(h=('0','0','sunny'),
                   x=('rainy','windy','cloudy'))

[('rainy', 'windy', '?')]

In [8]:
def min_specialization(h,domains,x):
  results =[]
  for i in range(len(h)):
    if h[i]=='?':
      for val in domains[i]:
        if x[i]!=val:
          h_new = h[:i]+(val,)+h[i+1:]
          results.append(h_new)
    elif h[i]!='0':
      h_new = h[:i]+('0',)+h[i+1:]
      results.append(h_new)
  return results

min_specialization(h=('?','x'),
                   domains=[['a','b','c'],['x','y']],
                   x=('b','x'))

[('a', 'x'), ('c', 'x'), ('?', '0')]

In [9]:
#with open('C:\\Users\\HP\\Downloads\\book1.csv') as csvFile:
  #examples = [tuple(line) for line in csv.reader(csvFile)]#

In [16]:
import random
import pandas as pd
import csv
import io
from google.colab import files
uploaded = files.upload()


Saving Book1.csv to Book1 (3).csv


In [17]:
example = csv.reader(io.StringIO(uploaded['Book1.csv'].decode('utf-8')))
example

In [18]:
examples = [tuple(line) for line in example]
examples

[('Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'same', 'Yes'),
 ('Sunny', 'Warm', 'High', 'Strong', 'Warm', 'same', 'Yes'),
 ('Rainy', 'Cold', 'High', 'Strong', 'Warm', 'change', 'No'),
 ('Sunny', 'Warm', 'High', 'Strong', 'cool', 'change', 'Yes')]

In [19]:
def get_domain(examples):
  d =[set() for i in examples[0]]
  for x in examples:
    for i,xi in enumerate(x):
      d[i].add(xi)

  return [list(sorted(x)) for x in d]

get_domain(examples)

[['Rainy', 'Sunny'],
 ['Cold', 'Warm'],
 ['High', 'Normal'],
 ['Strong'],
 ['Warm', 'cool'],
 ['change', 'same'],
 ['No', 'Yes']]

In [47]:
def candidate_elimination(examples):
  domain = get_domain(examples)[:-1]

  G = set([g_0(len(domain))])
  S = set([s_0(len(domain))])

  i=0

  print("\n G[{0}] :".format(i),G)
  print("\n S[{0}] :".format(i),S)

  for xcx in examples:
    i=i+1
    x,cx = xcx[:-1],xcx[-1]
    if cx=='Yes':
      G = {g for g in G if fulfills(x,g)}
      S = generalize_S(x,G,S)
    else:
      S = {s for s in S if not fulfills(x,s)}
      G = specialize_G(x,domain,G,S)
    
    print("\n G[{0}] :".format(i),G)
    print("\n S[{0}] :".format(i),S)
  return

In [48]:
def generalize_S(x,G,S):
  S_prev = list(S)
  for s in S_prev:
    if s not in S:
      continue
    if not fulfills(x,s):
      S.remove(s)
      Splus = min_generalization(s,x)
      S.update([h for h in Splus if any([more_general(g,h) for g in G ])])
      S.difference_update([h for h in S if any([more_general(h,h1) for h1 in S if h!=h1])])
  return S

def specialize_G(x,domain,G,S):
  G_prev = list(G)
  for g in G_prev:
    if g not in G:
      continue
    if fulfills(x,g):
      G.remove(g)
      Gminus = min_specialization(g,domain,x)
      G.update([h for h in Gminus if any([more_general(h, s) for s in S ])])
      G.difference_update([h for h in G if any([more_general(g1,h) for g1 in G if h!=g1])])
  return G


In [49]:
candidate_elimination(examples)


 G[0] : {('?', '?', '?', '?', '?', '?')}

 S[0] : {('0', '0', '0', '0', '0', '0')}

 G[1] : {('?', '?', '?', '?', '?', '?')}

 S[1] : {('Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'same')}

 G[2] : {('?', '?', '?', '?', '?', '?')}

 S[2] : {('Sunny', 'Warm', '?', 'Strong', 'Warm', 'same')}

 G[3] : {('?', 'Warm', '?', '?', '?', '?'), ('?', '?', '?', '?', '?', 'same'), ('Sunny', '?', '?', '?', '?', '?')}

 S[3] : {('Sunny', 'Warm', '?', 'Strong', 'Warm', 'same')}

 G[4] : {('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?')}

 S[4] : {('Sunny', 'Warm', '?', 'Strong', '?', '?')}
